# Adaptive Component Separation with K-means Clustering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CMBSciPol/furax-compsep-paper/blob/main/notebooks/kmeans_2.ipynb)

## Learning Objectives

By the end of this notebook, you will:
- Understand adaptive sky clustering for spatially-varying spectral parameters
- Implement spherical K-means clustering for CMB component separation
- Optimize parameters within each cluster using variance-based selection
- Visualize how spatial parameter variation improves foreground modeling

## The Adaptive Clustering Approach

Traditional component separation assumes uniform spectral parameters across the entire sky. In reality, Galactic emissions vary spatially. Our approach uses spherical K-means clustering to partition the sky into regions, allowing different spectral parameters in each cluster.

**Key Innovation**: Minimize CMB reconstruction variance by adaptively clustering sky pixels and optimizing spectral parameters per cluster.

In [1]:
# Setup and Data Loading

import os

os.environ["EQX_ON_ERROR"] = "nan"
os.environ["JAX_PLATFORM_NAME"] = "cpu"

# Core libraries
# Data utilities
import sys
from functools import partial

import healpy as hp

# JAX ecosystem
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import optax

# FURAX framework
from furax import HomothetyOperator
from furax.obs import negative_log_likelihood, sky_signal
from furax.obs.stokes import Stokes
from jax_grid_search import ProgressBar, optimize

# JAX-HEALPix for clustering and sky operations
from jax_healpy.clustering import (
    find_kmeans_clusters,
    get_cutout_from_mask,
    get_fullmap_from_cutout,
    normalize_by_first_occurrence,
)
from rich.progress import BarColumn, TimeElapsedColumn, TimeRemainingColumn
import scienceplots  # noqa: F401

sys.path.append("../data")
from generate_maps import get_mask, load_from_cache, save_to_cache

In [2]:
# Configure JAX
jax.config.update("jax_enable_x64", True)

# Load CMB and foreground data
nside = 64
npixels = 12 * nside**2

# Generate and load multi-frequency data
save_to_cache(nside, sky="c1d1s1", noise=False)
nu, freq_maps = load_from_cache(nside, sky="c1d1s1", noise=False)
print(f"Frequency maps shape: {freq_maps.shape}")
print(f"Frequencies: {len(nu)} bands from {nu[0]:.0f} to {nu[-1]:.0f} GHz")

# Convert to FURAX format (Q,U polarization)
d = Stokes.from_stokes(Q=freq_maps[:, 1, :], U=freq_maps[:, 2, :])

# Load galactic mask (cleanest 20% of sky)
mask = get_mask("GAL020", nside=nside)
(indices,) = jnp.where(mask == 1)
coverage = jnp.mean(mask) * 100
print(f"Sky coverage: {coverage:.1f}% ({len(indices):,} pixels)")

# Extract masked data for computation
masked_d = get_cutout_from_mask(d, indices, axis=1)
print(f"Masked data shape: {masked_d.shape}")

Generated and saved freq_maps for nside 64.
Loaded freq_maps for nside 64 from cache.
Frequency maps shape: (15, 3, 49152)
Frequencies: 15 bands from 40 to 402 GHz
Sky coverage: 19.7% (9,695 pixels)
Masked data shape: (15, 9695)


In [3]:
# Spherical K-means Clustering

# Configure clustering parameters for different spectral indices
cluster_config = {
    "temp_dust": 100,  # Dust temperature clusters
    "beta_dust": 2000,  # Dust spectral index clusters
    "beta_pl": 100,  # Synchrotron spectral index clusters
}

max_centroids = 2000  # Maximum cluster centers for algorithm

# Generate clusters for each parameter type
print("Generating spherical K-means clusters...")
clusters = {}

for param_name, cluster_count in cluster_config.items():
    print(f"  {param_name}: {cluster_count} clusters")

    # Generate spherical K-means clusters on unmasked pixels
    cluster_map = find_kmeans_clusters(
        mask,
        indices,
        cluster_count,
        jax.random.PRNGKey(42),  # Fixed seed for reproducibility
        max_centroids=max_centroids,
    )

    # Normalize cluster indices for consistent indexing
    cluster_map = normalize_by_first_occurrence(cluster_map, cluster_count, max_centroids)

    clusters[f"{param_name}_patches"] = cluster_map

print("Clustering complete!")

# Visualize the clustering results
fig = plt.figure(figsize=(15, 5))

param_labels = ["Dust Temperature", "Dust Spectral Index", "Synchrotron Index"]
for i, (param, label) in enumerate(zip(cluster_config.keys(), param_labels)):
    cluster_data = clusters[f"{param}_patches"]
    n_unique = jnp.unique(cluster_data[cluster_data != hp.UNSEEN]).size

    hp.mollview(
        cluster_data,
        title=f"{label}\\n({n_unique} clusters)",
        sub=(1, 3, i + 1),
        bgcolor=(0.0,) * 4,
    )

plt.tight_layout()
plt.show()

# Extract masked clusters for optimization
masked_clusters = get_cutout_from_mask(clusters, indices)
masked_clusters = jax.tree.map(lambda x: x.astype(jnp.int64), masked_clusters)
print("Masked clusters ready for optimization")

Generating spherical K-means clusters...
  temp_dust: 100 clusters
  beta_dust: 2000 clusters
  beta_pl: 100 clusters
Clustering complete!


/tmp/ipykernel_54759/1180964029.py:50: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Masked clusters ready for optimization


In [4]:
# Parameter Optimization

# Setup optimization parameters
dust_nu0 = 150.0  # Dust reference frequency (GHz)
synchrotron_nu0 = 20.0  # Synchrotron reference frequency (GHz)

# Create objective function with fixed reference frequencies
negative_log_likelihood_fn = partial(
    negative_log_likelihood, dust_nu0=dust_nu0, synchrotron_nu0=synchrotron_nu0
)

sky_signal_fn = partial(sky_signal, dust_nu0=dust_nu0, synchrotron_nu0=synchrotron_nu0)

# Initialize parameters for each cluster (realistic starting values)
initial_params = {
    "temp_dust": jnp.full((cluster_config["temp_dust"],), 20.0),  # 20K dust temperature
    "beta_dust": jnp.full((cluster_config["beta_dust"],), 1.54),  # Dust spectral index
    "beta_pl": jnp.full((cluster_config["beta_pl"],), -3.0),  # Synchrotron index
}

print("Parameter initialization:")
for param, values in initial_params.items():
    print(f"  {param}: {len(values)} clusters, initial value = {values[0]}")

# Create noise operator (simplified for demonstration)
N = HomothetyOperator(jnp.ones(1), _in_structure=masked_d.structure)

# Setup L-BFGS optimizer
solver = optax.lbfgs()

print("\\nRunning optimization...")
print("This may take a few minutes...")

# Progress bar setup
progress_columns = [
    "[progress.description]{task.description}",
    BarColumn(),
    "[progress.percentage]{task.percentage:>3.0f}%",
    TimeElapsedColumn(),
    TimeRemainingColumn(),
]

# Run optimization with progress bar
with ProgressBar(*progress_columns) as p:
    final_params, final_state = optimize(
        initial_params,
        negative_log_likelihood_fn,
        solver,
        max_iter=100,  # Reduced for demo
        tol=1e-6,
        progress=p,
        nu=nu,
        N=N,
        d=masked_d,
        patch_indices=masked_clusters,
    )

# Show optimization results
print("\\nOptimization completed!")
print(f"Iterations: {final_state.count}")
print(f"Final function value: {final_state.value:.2e}")

# Display optimized parameter statistics
print("\\nOptimized parameter ranges:")
for param, values in final_params.items():
    print(
        f"  {param}: [{jnp.min(values):.3f}, {jnp.max(values):.3f}], "
        f"mean = {jnp.mean(values):.3f} ± {jnp.std(values):.3f}"
    )

# Compute CMB reconstruction with optimized parameters
reconstructed_signal = sky_signal_fn(
    final_params, nu=nu, d=masked_d, N=N, patch_indices=masked_clusters
)
cmb_reconstruction = reconstructed_signal["cmb"]
print("\\nCMB reconstruction completed")
print(f"CMB shape: Q={cmb_reconstruction.q.shape}, U={cmb_reconstruction.u.shape}")

Output()

Parameter initialization:
  temp_dust: 100 clusters, initial value = 20.0
  beta_dust: 2000 clusters, initial value = 1.54
  beta_pl: 100 clusters, initial value = -3.0
\nRunning optimization...
This may take a few minutes...


\nOptimization completed!
Iterations: <built-in method count of OptimizerState object at 0x718cec8dd070>
Final function value: -2.02e+07
\nOptimized parameter ranges:
  beta_dust: [1.345, 1.775], mean = 1.632 ± 0.041
  beta_pl: [-3.136, -2.901], mean = -3.019 ± 0.047
  temp_dust: [19.231, 20.923], mean = 20.114 ± 0.205
\nCMB reconstruction completed
CMB shape: Q=(9695,), U=(9695,)


In [5]:
# Parameter Map Reconstruction

# Map cluster parameters back to full sky maps
print("Reconstructing full-sky parameter maps...")

param_maps = {}
for param_name in ["temp_dust", "beta_dust", "beta_pl"]:
    # Get optimized parameter values for each cluster
    param_values = final_params[param_name]
    cluster_indices = masked_clusters[f"{param_name}_patches"]

    # Map parameter values to masked pixels using cluster assignments
    param_map_masked = param_values[cluster_indices]

    # Convert back to full HEALPix map
    full_param_map = get_fullmap_from_cutout(param_map_masked, indices, nside=nside)
    param_maps[param_name] = full_param_map

print("Parameter map reconstruction completed!")

# Also reconstruct CMB maps for visualization
cmb_q_full = get_fullmap_from_cutout(cmb_reconstruction.q, indices, nside=nside)
cmb_u_full = get_fullmap_from_cutout(cmb_reconstruction.u, indices, nside=nside)

print("CMB maps reconstructed to full sky")
print(f"Parameter maps available: {list(param_maps.keys())}")

# Display parameter statistics
print("\\nFull-sky parameter statistics:")
for param_name, param_map in param_maps.items():
    valid_data = param_map[param_map != hp.UNSEEN]
    if len(valid_data) > 0:
        print(
            f"  {param_name}: [{jnp.min(valid_data):.3f}, {jnp.max(valid_data):.3f}], "
            f"mean = {jnp.mean(valid_data):.3f} ± {jnp.std(valid_data):.3f}"
        )
    else:
        print(f"  {param_name}: No valid data")

Reconstructing full-sky parameter maps...
Parameter map reconstruction completed!
CMB maps reconstructed to full sky
Parameter maps available: ['temp_dust', 'beta_dust', 'beta_pl']
\nFull-sky parameter statistics:
  temp_dust: [19.231, 20.923], mean = 20.115 ± 0.209
  beta_dust: [1.345, 1.775], mean = 1.632 ± 0.040
  beta_pl: [-3.136, -2.901], mean = -3.017 ± 0.047


In [6]:
# Results Visualization
fig = plt.figure(figsize=(15, 5))

# Plot the three parameter maps only
hp.mollview(param_maps["temp_dust"], title="Dust Temperature", sub=(1, 3, 1), bgcolor=(0.0,) * 4)

hp.mollview(param_maps["beta_dust"], title="Dust Spectral Index", sub=(1, 3, 2), bgcolor=(0.0,) * 4)

hp.mollview(param_maps["beta_pl"], title="Synchrotron Index", sub=(1, 3, 3), bgcolor=(0.0,) * 4)

plt.tight_layout()
plt.show()

/tmp/ipykernel_54759/1189568572.py:11: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## Command-Line Tool for Large-Scale Analysis

The complete workflow demonstrated above is implemented in the production script `08-KMeans-model.py` for large-scale analysis on HPC clusters.

### Basic Usage

```bash
# Run adaptive K-means clustering
cd content/
python 08-KMeans-model.py -n 64 -pc 1000 50 10 -tag c1d1s1 -m GAL020 -i LiteBIRD
```

### Key Parameters

- `-n 64`: HEALPix resolution (nside=64 → ~55 arcmin pixels)
- `-pc 1000 50 10`: Cluster counts for [dust_beta, dust_temp, sync_beta]
- `-tag c1d1s1`: Sky simulation (CMB + dust + synchrotron)
- `-m GAL020`: Galactic mask (20% cleanest sky)
- `-i LiteBIRD`: Instrument configuration

### Output Structure

Results are saved to `results/kmeans_{config}/`:
- `best_params.npz`: Optimized spectral parameters per cluster
- `results.npz`: Full clustering and optimization results  
- `mask.npy`: Sky mask used for analysis

### Scaling to Large Problems

The command-line tool provides:
- **Distributed computing**: SLURM integration for GPU clusters
- **Monte Carlo analysis**: Multiple noise realizations (`-ns` parameter)
- **Parameter bounds**: Physical constraints on spectral parameters
- **Grid search**: Systematic exploration of cluster configurations
- **Variance-based selection**: Automatic optimal cluster determination

## Key Benefits

1. **Spatial Adaptivity**: Parameters vary across the sky to match astrophysical reality
2. **Variance Minimization**: Optimizes CMB reconstruction quality directly  
3. **Scalable Implementation**: JAX enables GPU acceleration and distributed computing
4. **Physical Constraints**: Realistic parameter bounds ensure meaningful results

This adaptive clustering approach provides significant improvements over uniform parameter methods, particularly for constraining the tensor-to-scalar ratio r in CMB B-mode analysis.